In [ ]:
%%html
<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To hide/show code, click <a href="javascript:code_toggle()">here</a>.

In [ ]:
#DataBase Connection
from sqlalchemy import create_engine, Column, Table, MetaData
from sqlalchemy.sql import text

#Dataframe importing, series usage, etc.
import pandas as pd
#Widgets management
import traitlets
import ipywidgets as widgets
from tkinter import Tk, filedialog
from IPython.core.display import clear_output, display, HTML
display(HTML(open('ipython.css','r').read()))

In [ ]:
#Dataframe connection
def db_engine(dsn_hostname='web566.webfaction.com', dsn_port='5432',
              dsn_database='analytics', dsn_uid='analytic',
              dsn_pwd='analytic#32'):
    """ """
    try:
        conn_string = r"postgresql+psycopg2://{}:{}@{}:{}/{}".format(
            dsn_uid, dsn_pwd, dsn_hostname, dsn_port, dsn_database)
        engine = create_engine(conn_string)
        return engine
    except:
        return None

In [ ]:
#Create Dataframe Connection Engine
Conection_engine = db_engine()

In [ ]:
class Display:
    # Class that makes a display for the user
    def __init__(self):
        # Initialization of class. Spawns the needed widgets
        
        # Excel loading button
        self._bsf = widgets.Button(layout=widgets.Layout(width='200px',height='50px'))
        self._bsf.add_traits(files=traitlets.traitlets.List())
        self._bsf.description = "Select File"
        self._bsf.icon = "square-o"
        self._bsf.on_click(self.select_excel)
                
        #Submit data button
        self._bds = widgets.Button(layout=widgets.Layout(width='200px',height='50px'))
        self._bds.description = "Create Table"
        self._bds.on_click(self.submit_data)
        self._bds.disabled = True
        
        # Data printing button
        self._bdp = widgets.Button(layout=widgets.Layout(width='200px',height='50px'))
        self._bdp.description = "Show Data"
        self._bdp.on_click(self.print_data)
        self._bdp.disabled = True
        
        # Name of table text area
        self._tnt = widgets.Text(value='',
                                 placeholder='Insert Table Name',
                                 description='Table name:',
                                 disabled=False)
        
        # Table selection droplist
        style = {'description_width': 'initial'}
        self._dds = widgets.Dropdown(options=['Select file'], value='Select file', disabled=True, style=style,
                                     description='Spreadsheet:')
        
        # Observe changes in the value of the list
        self._dds.observe(self.choose_spreadsheet, names='value', type='change')
        
        # Observe changes in the value of text
        self._tnt.observe(self.enable_table_creation, names='value', type='change')
        
        #Output
        self._output = widgets.Output()
        
        # Layout
        self.box_layout = widgets.Layout(display='center',
                            flex_flow='row',
                            align_items='center',
                            border='None',
                            justify_content='space-between',
                            width='900px')   
        self._box = widgets.Box(children=[self._bsf,self._bdp, self._dds], layout=self.box_layout, main_size = 10)
        self._secondbox = widgets.Box(children=[self._bds, self._tnt])
        display(self._box)
        display(self._output)
        self._vbox = widgets.VBox()
        self.chosen_list = []
        
    def select_excel(self, b): 
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        #File selection screen
        new_file = filedialog.askopenfilename(multiple=True,filetypes = [("XLS","*.xls"), ("XLSX", "*.xlsx")])
        
        if new_file != "":
            self._bsf.files = new_file
            self._bsf.description = "File Selected"
            self._bsf.icon = "check-square-o"
            self._bsf.button_style = "success"
            #self._blj.disabled = False
            #print('Selected file: {}'.format(self._bsf.files))
            file_path = self._bsf.files
            self._excel = pd.ExcelFile(file_path[0], on_demand = True)
            self._sheets = self._excel.sheet_names
            self._dds.disabled = False
            self._dds.options = ['↓ Select spreadsheet ↓'] + self._sheets 
            self._dds.value = '↓ Select spreadsheet ↓'
            
            self.load_excel
    
    def load_excel(self, b):
        self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value) 
        self._blj.button_style = "success"
        display(self._dataframe.head())
        self._csm.disabled = False
        self._csm.options = list(self._dataframe.columns.values)
    

    def choose_spreadsheet(self, b):
        if self._dds.value in self._sheets :
            with self._output:
                self._output.clear_output()
                display(HTML("""<div class="sk-circle">
                         <div class="sk-circle1 sk-child"></div>
                         <div class="sk-circle2 sk-child"></div>
                         <div class="sk-circle3 sk-child"></div>
                         <div class="sk-circle4 sk-child"></div>
                         <div class="sk-circle5 sk-child"></div>
                         <div class="sk-circle6 sk-child"></div>
                         <div class="sk-circle7 sk-child"></div>
                         <div class="sk-circle8 sk-child"></div>
                         <div class="sk-circle9 sk-child"></div>
                         <div class="sk-circle10 sk-child"></div>
                         <div class="sk-circle11 sk-child"></div>
                         <div class="sk-circle12 sk-child"></div>
                         </div>"""))
                clear_output(wait=True)
            self._dataframe = pd.read_excel(self._bsf.files[0], sheet_name = self._dds.value)
            self._bdp.disabled = False
            self.column_names = list(self._dataframe.columns.values)
            self.generate_droplists()
    
    def generate_droplists(self):

        self.droptable_list = []
        self.hbox_list = []
        self.droptable_options = pd.read_sql_table('reliability', Conection_engine).columns.values.tolist()
        for index, element in enumerate(self.column_names):
            self.droptable_list.append(widgets.Dropdown(options=['No Column Atributed']+self.droptable_options,
                                                        value='No Column Atributed',description=element,))
            if index % 4 == 3:
                self.hbox_list.append(widgets.HBox(self.droptable_list))
                self.droptable_list = []
        
        
        self.hbox_list.append(widgets.HBox(self.droptable_list))

        self._vbox = widgets.VBox(self.hbox_list)
        
        for hbox in self._vbox.children:
            for droptable in hbox.children:
                droptable.observe(self.refresh_droplists, names='value')
        self._tnt.value = self._dds.value
        display(self._box)
        with self._output:
            self._output.clear_output()
            display(self._vbox)
            display(self._secondbox)
        
    def print_data(self, b):
        for element in self.column_names:
            display(self._dataframe[element].head())
        self._bdp.button_style = "success"
        self._bdp.description = "Data Shown"
        self._bdp.disabled = True
        
    def refresh_droplists(self, b):
        self.chosen_list = list()
        for hbox in self._vbox.children:
            for droplist in hbox.children:
                droplist.unobserve(self.refresh_droplists, names='value')
                if droplist.value != 'No Column Atributed':
                    self.chosen_list.append(droplist.value)
        self.unchosen_list = ['No Column Atributed']+[element for element in self.droptable_options if element not in self.chosen_list]
        
        for hbox in self._vbox.children:
            for droplist in hbox.children:
                if droplist.value == 'No Column Atributed':
                    droplist.options = self.unchosen_list
                else:
                    droplist.options = [droplist.value] + self.unchosen_list
        for hbox in self._vbox.children:
            for droptable in hbox.children:
                droptable.observe(self.refresh_droplists, names='value')
        
    def enable_table_creation(self, b):
        if self._tnt.value == '':
            self._bds.disabled = True
        else:
            self._bds.disabled = False
            
    def submit_data(self, b):
        self._switcheroo_dict = {}
        self._prepared_df = self._dataframe
        for hbox in self._vbox.children:
            for droplist in hbox.children:
                if droplist.value != 'No Column Atributed':
                    self._switcheroo_dict[droplist.description] = droplist.value
                else:
                    del self._prepared_df[droplist.description]
        self._prepared_df = self._prepared_df.rename(columns=self._switcheroo_dict)
        self._table_name = 'wo_' + self._tnt.value
        #self._prepared_df.to_sql(self._table_name, Conection_engine)

In [ ]:
d = Display()

In [ ]:
dataframe = d._prepared_df[:23408]

In [ ]:
len(dataframe)

In [ ]:
dataframe['clientid'] = [4 for i in range(23408)]

In [ ]:
dataframe[isinstance(dataframe.wo_total_cost.values(), str)]

In [ ]:
dataframe.to_sql('reliability', Conection_engine, if_exists='append', index=False)

In [ ]:
print(dataframe.wo_total_cost.values == str)

In [ ]:
aux_list = list()
for element in dataframe.wo_total_cost.values:
    if type(element) == str:
        element = float(element[:-1].replace(',',''))
        print(element)
        aux_list.append(element)
    else:
        aux_list.append(element)

In [ ]:
aux_list

In [ ]:
del dataframe['wo_total_cost']
dataframe['wo_total_cost'] = aux_list

In [ ]:
dataframe

In [ ]:
None

In [ ]:
widgets.Button

In [ ]:
X = 'Button'

In [ ]:
getattr(widgets,X)